# Comparison of the efficiency of the MIHA algorithm with the Optuna framework

In [18]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna

from matplotlib import pyplot as plt
from matplotlib import cm

import warnings
warnings.filterwarnings("ignore")

## Model obtained using the MIHA algorithm

The number of epochs produced that it took to train the model is 176.

In [38]:
from torch import jit

net = jit.load('D:/miha_exp/mnist/4/model/model_final.zip')
state = torch.load('D:/miha_exp/mnist/4/model/optimizer_final.pth')

# Uploading 10 000 MNIST objects for training
X_train = torch.load('D:/ITMO/Neural_network_example/mnist/X_train.pt')
Y_train = torch.load('D:/ITMO/Neural_network_example/mnist/Y_train.pt')

# And for testing
X_test = torch.load('D:/ITMO/Neural_network_example/mnist/X_test.pt')
Y_test = torch.load('D:/ITMO/Neural_network_example/mnist/Y_test.pt')

# We check accuracy on 500 objects
X_test = X_test[:500, :]
Y_test = Y_test[:500, ]

train = torch.utils.data.TensorDataset(X_train, Y_train)
test = torch.utils.data.TensorDataset(X_test, Y_test)

# Prepare data loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=20, num_workers=0)
test_loader = torch.utils.data.DataLoader(test, batch_size=20, num_workers=0)

Let's check the accuracy of the model on the test dataset.

In [39]:
def validation(net, test_loader):
    net.eval()
    
    true_labels = []
    predicted_labels = []
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # The probability of assignment to classes
            output = net(data)
            # Predicted labels
            pred = output.argmax(dim=1, keepdim=False)
            
            # Number of correctly predicted class labels
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # Transform to lists
            target = list(map(int, target))
            pred = list(map(int, pred))
            
            true_labels.extend(list(target))
            predicted_labels.extend(list(pred))

        accuracy = correct / len(test_loader.dataset)
   
    print(f'Accuracy - {accuracy}')
    print(f'{correct} labels out of {len(test_loader.dataset)} objects were correctly assigned')
    
validation(net = net, test_loader = test_loader)

Accuracy - 0.974
487 labels out of 500 objects were correctly assigned


Thus, the prepared neural network made a mistake only on 13 objects.

## Feedforward neural network (FNN) which was optimized by Optuna 

Optimization using the optuna framework. The code below is a modified version of the official example, see [here](https://github.com/optuna/optuna/blob/master/examples/pytorch_simple.py).

In [9]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
test_accuracy = 0

def define_model(trial):
    
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))
    
    return nn.Sequential(*layers)

def objective(trial):
    global test_accuracy

    produced_epochs = 0
    
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Training of the model.
    for epoch in range(EPOCHS):
        correct = 0
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            
            pred = output.argmax(dim=1, keepdim=False)
            correct += pred.eq(target.view_as(pred)).sum().item()
                
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
        
        train_accuracy = correct / len(train_loader.dataset)
        produced_epochs += 1
        
        # Validation of the model
        model.eval()
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                pred = output.argmax(dim=1, keepdim=False)
                correct += pred.eq(target.view_as(pred)).sum().item()

        new_test_accuracy = correct / len(test_loader.dataset)
        
        # Update test accuracy
        if new_test_accuracy > test_accuracy:
            test_accuracy = new_test_accuracy
        
        trial.report(train_accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    print(f'Number of epochs produced {produced_epochs}')
    return train_accuracy

def make_test_validation(best_model):
    
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Вероятности отнесения к классам
            output = best_model(data)
            # Предсказанные метки
            pred = output.argmax(dim=1, keepdim=False)
            
            # Количество правильно предсказанных меток класса
            correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(test_loader.dataset)
   
    print(f'Accuracy - {accuracy}')
    print(f'{correct} labels out of {len(test_loader.dataset)} objects were correctly assigned')
    

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, timeout=600)

    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial
    
    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        
    print(f'\nBest model accuracy {test_accuracy}')

[I 2021-01-15 22:32:49,129] A new study created in memory with name: no-name-9a9f59f3-225b-4267-a0fe-660d351169b6
[I 2021-01-15 22:33:07,805] Trial 0 finished with value: 0.9098 and parameters: {'n_layers': 3, 'n_units_l0': 429, 'dropout_l0': 0.41084891219713937, 'n_units_l1': 85, 'dropout_l1': 0.22274557247090618, 'n_units_l2': 276, 'dropout_l2': 0.25610214111637236, 'optimizer': 'Adam', 'lr': 5.736254519743909e-05}. Best is trial 0 with value: 0.9098.


Number of epochs produced 10


[I 2021-01-15 22:33:16,182] Trial 1 finished with value: 0.2741 and parameters: {'n_layers': 3, 'n_units_l0': 64, 'dropout_l0': 0.2297348311081977, 'n_units_l1': 328, 'dropout_l1': 0.33050124281172644, 'n_units_l2': 483, 'dropout_l2': 0.3402349211188188, 'optimizer': 'SGD', 'lr': 0.0007298508269815263}. Best is trial 0 with value: 0.9098.


Number of epochs produced 10


[I 2021-01-15 22:33:19,942] Trial 2 finished with value: 0.3112 and parameters: {'n_layers': 1, 'n_units_l0': 94, 'dropout_l0': 0.34956241407144784, 'optimizer': 'SGD', 'lr': 8.472899768329822e-05}. Best is trial 0 with value: 0.9098.


...

Number of epochs produced 10


[I 2021-01-15 22:40:25,314] Trial 47 finished with value: 0.9877 and parameters: {'n_layers': 1, 'n_units_l0': 393, 'dropout_l0': 0.2631760709804497, 'optimizer': 'Adam', 'lr': 0.0015300896321639996}. Best is trial 25 with value: 0.9903.


Number of epochs produced 10


[I 2021-01-15 22:40:27,059] Trial 48 pruned. 
[I 2021-01-15 22:40:28,669] Trial 49 pruned. 


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  20
  Number of complete trials:  30
Best trial:
  Value:  0.9903
  Params: 
    n_layers: 2
    n_units_l0: 497
    dropout_l0: 0.20944805967030697
    n_units_l1: 352
    dropout_l1: 0.45447755536395607
    optimizer: RMSprop
    lr: 0.0005672680824260684

Best model accuracy 0.976


#### As you can see, the model obtained using the MIHA algorithm gave a similar result (accuracy 0.974) to the neural network optimized using Optuna (accuracy 0.976)

But the repository with Optuna has several thousand stars, and this library has none, change it (We really tried very hard)!